5CS037 - Concepts and Technologies of AI.

Decision Tree for Classification and Regression.
Decision Tree, Ensemble Methods and Hyperparameter
Optimizations with Gridsearch and Cross Validation

**Important Imports:**

In [ ]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

Custom Vs. Scikit Learn Built Decision Tree.

1. Step -1- Building a Custom Decision Tree with Information Gain:

In [ ]:
import numpy as np

class CustomDecisionTree:
    def __init__(self, max_depth=None):
        self.max_depth = max_depth
        self.tree = None

    def fit(self, X, y):
        self.tree = self._build_tree(X, y)

    def _build_tree(self, X, y, depth=0):
        num_samples, num_features = X.shape
        unique_classes = np.unique(y)

        # Stop if pure node
        if len(unique_classes) == 1:
            return {"class": unique_classes[0]}

        # Stop if max depth reached
        if self.max_depth is not None and depth >= self.max_depth:
            return {"class": np.bincount(y).argmax()}

        best_gain = -1
        best_feature = None
        best_threshold = None
        best_left_idx = None
        best_right_idx = None

        for feature_idx in range(num_features):
            thresholds = np.unique(X[:, feature_idx])

            for threshold in thresholds:
                left_idx = X[:, feature_idx] <= threshold
                right_idx = X[:, feature_idx] > threshold

                if len(y[left_idx]) == 0 or len(y[right_idx]) == 0:
                    continue

                gain = self._information_gain(y, y[left_idx], y[right_idx])

                if gain > best_gain:
                    best_gain = gain
                    best_feature = feature_idx
                    best_threshold = threshold
                    best_left_idx = left_idx
                    best_right_idx = right_idx

        if best_gain == -1:
            return {"class": np.bincount(y).argmax()}

        left_tree = self._build_tree(
            X[best_left_idx], y[best_left_idx], depth + 1
        )
        right_tree = self._build_tree(
            X[best_right_idx], y[best_right_idx], depth + 1
        )

        return {
            "feature_idx": best_feature,
            "threshold": best_threshold,
            "left_tree": left_tree,
            "right_tree": right_tree
        }

    def _information_gain(self, parent, left, right):
        parent_entropy = self._entropy(parent)
        left_entropy = self._entropy(left)
        right_entropy = self._entropy(right)

        weighted_entropy = (
            (len(left) / len(parent)) * left_entropy +
            (len(right) / len(parent)) * right_entropy
        )

        return parent_entropy - weighted_entropy

    def _entropy(self, y):
        probs = np.bincount(y) / len(y)
        return -np.sum(probs * np.log2(probs + 1e-9))

    def predict(self, X):
        return np.array([self._predict_single(x, self.tree) for x in X])

    def _predict_single(self, x, tree):
        if "class" in tree:
            return tree["class"]

        if x[tree["feature_idx"]] <= tree["threshold"]:
            return self._predict_single(x, tree["left_tree"])
        else:
            return self._predict_single(x, tree["right_tree"])


Step -2- Load and Split the Iris Datasets:
Load and Split the IRIS Dataset:

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

data = load_iris()
X = data.data
y = data.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


Step -3- Train and Evaluate a Custom Decision Tree:
Train and Evaluate a Custom Decision Tree:

In [ ]:
from sklearn.metrics import accuracy_score

custom_tree = CustomDecisionTree(max_depth=3)
custom_tree.fit(X_train, y_train)

y_pred_custom = custom_tree.predict(X_test)
accuracy_custom = accuracy_score(y_test, y_pred_custom)

print(f"Custom Decision Tree Accuracy: {accuracy_custom:.4f}")


Custom Decision Tree Accuracy: 1.0000


Step -4- Train and Evaluate a Scikit Learn Decision Tree:
Train and Evaluate a Scikit Learn Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier

sklearn_tree = DecisionTreeClassifier(max_depth=3, random_state=42)
sklearn_tree.fit(X_train, y_train)

y_pred_sklearn = sklearn_tree.predict(X_test)
accuracy_sklearn = accuracy_score(y_test, y_pred_sklearn)

print(f"Scikit-learn Decision Tree Accuracy: {accuracy_sklearn:.4f}")


Scikit-learn Decision Tree Accuracy: 1.0000


Step -5- Result Comparison:

In [ ]:
print("Accuracy Comparison:")
print(f"Custom Decision Tree: {accuracy_custom:.4f}")
print(f"Scikit-learn Decision Tree: {accuracy_sklearn:.4f}")


Accuracy Comparison:
Custom Decision Tree: 1.0000
Scikit-learn Decision Tree: 1.0000


3 Ensemble Methods and Hyperparameter Tuning:
1. Implement Classification Models:


In [ ]:
import numpy as np

from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import f1_score

from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor


In [ ]:
wine = load_wine()
X = wine.data
y = wine.target

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)


In [ ]:
dt_clf = DecisionTreeClassifier(random_state=42)
dt_clf.fit(X_train, y_train)

y_pred_dt = dt_clf.predict(X_test)
f1_dt = f1_score(y_test, y_pred_dt, average="weighted")

print(f"Decision Tree F1 Score: {f1_dt:.4f}")


Decision Tree F1 Score: 0.9632


In [ ]:
rf_clf = RandomForestClassifier(random_state=42)
rf_clf.fit(X_train, y_train)

y_pred_rf = rf_clf.predict(X_test)
f1_rf = f1_score(y_test, y_pred_rf, average="weighted")

print(f"Random Forest F1 Score: {f1_rf:.4f}")


Random Forest F1 Score: 1.0000


2. Hyperparameter Tuning:

In [ ]:
param_grid = {
    "n_estimators": [50, 100, 200],
    "max_depth": [None, 5, 10],
    "min_samples_split": [2, 5, 10]
}

grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=param_grid,
    scoring="f1_weighted",
    cv=5,
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

best_rf_clf = grid_search.best_estimator_

y_pred_best = best_rf_clf.predict(X_test)
best_f1 = f1_score(y_test, y_pred_best, average="weighted")

print("Best Parameters:", grid_search.best_params_)
print(f"Tuned Random Forest F1 Score: {best_f1:.4f}")


Best Parameters: {'max_depth': None, 'min_samples_split': 2, 'n_estimators': 200}
Tuned Random Forest F1 Score: 1.0000


3. Implement Regression Model:

In [ ]:
X_reg = X[:, 1:]   # Features
y_reg = X[:, 0]    # Alcohol (target)

X_train_r, X_test_r, y_train_r, y_test_r = train_test_split(
    X_reg, y_reg, test_size=0.3, random_state=42
)


In [ ]:
dt_reg = DecisionTreeRegressor(random_state=42)
dt_reg.fit(X_train_r, y_train_r)

dt_score = dt_reg.score(X_test_r, y_test_r)
print(f"Decision Tree Regressor R² Score: {dt_score:.4f}")


Decision Tree Regressor R² Score: 0.3399


In [ ]:
rf_reg = RandomForestRegressor(random_state=42)
rf_reg.fit(X_train_r, y_train_r)

rf_score = rf_reg.score(X_test_r, y_test_r)
print(f"Random Forest Regressor R² Score: {rf_score:.4f}")


Random Forest Regressor R² Score: 0.6447


In [ ]:
param_dist = {
    "n_estimators": [50, 100, 200, 300],
    "max_depth": [None, 5, 10, 15],
    "min_samples_split": [2, 5, 10]
}

random_search = RandomizedSearchCV(
    estimator=RandomForestRegressor(random_state=42),
    param_distributions=param_dist,
    n_iter=10,
    cv=5,
    scoring="r2",
    random_state=42,
    n_jobs=-1
)

random_search.fit(X_train_r, y_train_r)

best_rf_reg = random_search.best_estimator_
best_reg_score = best_rf_reg.score(X_test_r, y_test_r)

print("Best Regression Parameters:", random_search.best_params_)
print(f"Tuned Random Forest Regressor R² Score: {best_reg_score:.4f}")


Best Regression Parameters: {'n_estimators': 50, 'min_samples_split': 2, 'max_depth': 5}
Tuned Random Forest Regressor R² Score: 0.6576
